In [0]:
from pyspark.sql.functions import current_timestamp, date_format, from_utc_timestamp

In [0]:
input_path = "dbfs:/Volumes/workspace/anac/silver/anac_silver.parquet/"
output_path = "/Volumes/workspace/anac/gold"

In [0]:
df = (spark.read.parquet(input_path))

colunas = [
      'Aerodromo_de_Destino'
    , 'Aerodromo_de_Origem'
    , 'Classificacao_da_Ocorrência'
    , 'Danos_a_Aeronave'
    , 'Data_da_Ocorrencia'
    , 'Municipio'
    , 'UF'
    , 'Regiao'
    , 'Tipo_de_Aerodromo'
    , 'Tipo_de_Ocorrencia'
    , 'Lesoes_Desconhecidas_Passageiros'
    , 'Lesoes_Desconhecidas_Terceiros'
    , 'Lesoes_Desconhecidas_Tripulantes'
    , 'Lesoes_Fatais_Passageiros'
    , 'Lesoes_Fatais_Terceiros'
    , 'Lesoes_Fatais_Tripulantes'
    , 'Lesoes_Graves_Passageiros'
    , 'Lesoes_Graves_Terceiros'
    , 'Lesoes_Graves_Tripulantes'
    , 'Lesoes_Leves_Passageiros'
    , 'Lesoes_Leves_Terceiros'
    , 'Lesoes_Leves_Tripulantes'
]

df = df.select(colunas)

In [0]:
colunas_a_somar = [
      'Lesoes_Desconhecidas_Passageiros'
    , 'Lesoes_Desconhecidas_Terceiros'
    , 'Lesoes_Desconhecidas_Tripulantes'
    , 'Lesoes_Fatais_Passageiros'
    , 'Lesoes_Fatais_Terceiros'
    , 'Lesoes_Fatais_Tripulantes'
    , 'Lesoes_Graves_Passageiros'
    , 'Lesoes_Graves_Terceiros'
    , 'Lesoes_Graves_Tripulantes'
    , 'Lesoes_Leves_Passageiros'
    , 'Lesoes_Leves_Terceiros'
    , 'Lesoes_Leves_Tripulantes'
]

df = (df
      .withColumn("Total_Lesoes", sum(df[somar_colunas] for somar_colunas in colunas_a_somar)))

In [0]:
df = (df
      .withColumnRenamed('Aerodromo_de_Destino', 'Destino')
      .withColumnRenamed('Aerodromo_de_Origem', 'Origem')
      .withColumnRenamed('Classificacao_da_Ocorrência', 'Classificacao')
      .withColumnRenamed('Danos_a_Aeronave', 'Danos')
      .withColumnRenamed('Data_da_Ocorrencia', 'Data')
      .withColumnRenamed('UF', 'Estado')
      .withColumnRenamed('Regiao', 'Regiao')
      .withColumn("Atualização",
                  date_format(from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"), "yyyy-MM-dd HH:mm:ss")))

In [0]:
classificacao_a_excluir = ["Exterior", "null", "Indeterminado"]
df = (df.filter(~df.Estado.isin(classificacao_a_excluir)))

In [0]:
(df
 .write
 .mode("overwrite")
 .format("parquet")
 .partitionBy("Estado")
 .save(output_path + "/anac_gold_particionado.parquet"))